In [ ]:
import numpy as np
import os
from Curvas import Curve
from Curvas import import_measured_curves
from typing import List
import copy

measurement_file_path = r'P:\8 - Físicos Médicos\Roy\INTECNUS---RoyQA\Medido Synergy 2024\electrones synery 2024.asc'
monaco_calculation_file_path = r'P:\8 - Físicos Médicos\Roy\INTECNUS---RoyQA\MONACO'

def import_calculated_curves(monaco_calculation_file_path, machine=''):

    def get_energy_from_filename(file_name):
        electron_energies = ['06', '09', '12', '15']
        file_extension_index = int(file_name.split('.')[::-1][0]) - 1
        return int(electron_energies[file_extension_index])

    data = np.genfromtxt(monaco_calculation_file_path, dtype=str, delimiter='\n', skip_header=0, skip_footer=0)
    metadata = data[:16]
    data = data[16:]

    curves = []
    curve1 = Curve()
    curve1.type = 'C'
    curve1.machine = machine

    try:
        if 'ELECTRONES' in monaco_calculation_file_path:
            curve1.particle = 1   #electrones
            # Attempt to unpack the last 6 elements after reversing the split result
            file_name, curve1.field_size, curve1.SSD, _ = monaco_calculation_file_path.split('/')[::-1][:5]
            curve1.energy = get_energy_from_filename(file_name)
            
            # Ensure that exactly 6 variables are unpacked (including the placeholder '_')
            assert len(monaco_calculation_file_path.split('/')[::-1][:5]) == 5
        else:
            curve1.particle = 0   #fotones

            file_name, curve1.field_size,curve1. SSD, _, curve1.energy = monaco_calculation_file_path.split('/')[::-1][:6]
            
            assert len(monaco_calculation_file_path.split('/')[::-1][:6]) == 6

    except ValueError:
        # This block runs if the unpacking fails due to a mismatch in the number of elements
        print("Unpacking failed: The number of elements in the string does not match the expected 6.")
        return []
    
    except AssertionError:
        # This block runs if the assertion fails, meaning there are not enough elements
        print("Error: Not enough elements to unpack. Expected 6 elements.")
        return []


    _, dose_plane, depth, _ = file_name.split('.')

    if dose_plane == 'Coronal':
        #Depth:
        z = str(250 - int(depth))
        if len(z) < 3:
            curve1.depth = '0' + z
        else:
            curve1.depth = z
        
        # Aca creo dos perfiles porque el txt tiene la informacion de dos curvas segun si se exporto coronal o transversal
        dose2D = np.array([row.split(',') for row in data])
        dose2D = dose2D.astype('float32')

        spatial_resolution = float(metadata[15].split(',')[-1])

        # Cargo coordinate, axis y relative_dose del perfil en X
        curve1.coordinate    = 'X' 
        curve1.axis          = np.arange(-dose2D.shape[1] // 2, dose2D.shape[1] // 2, spatial_resolution)
        curve1.relative_dose = dose2D[dose2D.shape[0] // 2, :]

        curves.append(curve1)

        # Cargo coordinate, axis y relative_dose del perfil en Y
        curve2 = copy.copy(curve1)

        curve2.coordinate    = 'Y' 
        curve2.axis          = np.arange(-dose2D.shape[0] // 2, dose2D.shape[0] // 2, spatial_resolution)
        curve2.relative_dose = dose2D[:, dose2D.shape[1] // 2]

        curves.append(curve2)

    if dose_plane == 'Transverse':
        # Cargo coordinate, axis y relative_dose del PDD

        curve1.coordinate    = 'Z' 
        curve1.axis          = np.arange(0,dose2D.shape[0]//2+spatial_resolution,spatial_resolution)
        curve1.relative_dose = dose2D[dose2D.shape[0]//2:,dose2D.shape[1]//2]

        curves.append(curve1)   

    return curves

meas_curves = import_measured_curves(measurement_file_path, machine='P')
calc_curves = import_calculated_curves(monaco_calculation_file_path, machine='P')
